In [ ]:
# This examples shows who to download files from the ONC server
import glob
import os

import numpy as np
import datetime
import scipy

import strawb
import strawb.sensors.lidar
import strawb.tools

import plotly
import plotly.graph_objs as go
import plotly.express as px
import pandas
import plotly.io as pio
pio.templates.default = "plotly_white"

import random

## Get the right files

In [ ]:
def get_working_file_randomly(file_list, file_version=1, **file_attributes):
    """Some files fails with the file_handler, i.e., if a group is missing for older SDAQ versions.
    To get one file randomly which works, try it until it works.
    """

    success = False
    file_list_trials = file_list.copy()
    while len(file_list_trials) > 0 and not success:
        i = np.random.randint(0,len(file_list_trials))
        full_path = file_list_trials[i]
        try:
            lidar = strawb.sensors.lidar.FileHandler(full_path)
        except Exception as a:
            print(f'Error: {full_path} failed with: {a}')
        else:
            print(f'Success: {full_path}, file_version={lidar.file_version}, len={len(file_list_trials)}')
            if lidar.file_version >= file_version:
                print(f' file_version match')
                # check if all file_attributes are available
                if set(file_attributes).difference(lidar.file_attributes) == set():
                    # check if all values match
                    same_values = [lidar.file_attributes[i] == file_attributes[i] for i in file_attributes]
                    print(f' file_attributes match: {same_values}')
                    if same_values == [] or any(same_values):
                        print(f'Use: {full_path}')
                        return full_path
                    
        file_list_trials.pop(i)
            
    return None
        

In [ ]:
# Get all files available
# takes one file randomly, takes some seconds
file_list = strawb.sensors.lidar.FileHandler.find_files_glob('*-SDAQ-LIDAR.hdf5',
                                           directory=strawb.Config.raw_data_dir,
                                           recursive=True,
                                           raise_nothing_found=True)

In [ ]:
# in case execute db.load_entire_db_from_ONC() to load the entire db
db = strawb.SyncDBHandler(file_name='Default')  # loads the db

# mask by device
mask = db.dataframe['deviceCode'] == 'TUMLIDAR001'
mask |= db.dataframe['deviceCode'] == 'TUMLIDAR002'

# and add mask by 'measurement_type'
mask &= db.dataframe['measurement_type'] == 'laser_adjustment'

db.dataframe[mask]  # plot the table

In [ ]:
file_version = 3  # for LiDARs select one of [1, 2, 3]

# select a random file
full_path = get_working_file_randomly(file_list,
                                      file_version,
                                      mes_typ='laser_adjustment',  # or other file_handler.file_attributes
                                     )

## Load files

In [ ]:
# 2 files of the same scan on both lidars
file_lidar1 = strawb.Config.raw_data_dir + '/tumlidar001/2021_10/TUMLIDAR001_20211011T160751.622Z-SDAQ-LIDAR.hdf5'
file_lidar2 = strawb.Config.raw_data_dir + '/tumlidar002/2021_10/TUMLIDAR002_20211011T163516.953Z-SDAQ-LIDAR.hdf5'

lidar1 = strawb.sensors.lidar.Lidar(file=file_lidar1)
print(lidar1.file_handler.file_attributes)  # show the file attributes, time in seconds since epoch (1.1.1970) UTC

lidar2 = strawb.sensors.lidar.Lidar(file=file_lidar2)
lidar2.file_handler.file_attributes  # show the file attributes, time in seconds since epoch (1.1.1970) UTC

# select the device to use for the following

In [ ]:
lidar = lidar1
#lidar = lidar2

# Set up plotting function

In [ ]:
def multiline_plot(data_frame):
    columns = list(set(data_frame.columns).difference(['time']))
    
    fig = plotly.subplots.make_subplots(shared_xaxes=True,
                                        rows=len(columns),
                                        cols=1, 
                                        #subplot_titles=columns
                                       )


    fig.update_layout(
        width=800,
        height=150+100*len(columns),)

    j = 1
    for i, column_i in enumerate(columns):
        fig.add_trace(go.Scatter(x=data_frame['time'], 
                                 y=data_frame[column_i],
                                 name=column_i), 
                      row=i+1, col=1
                     )

    return fig

# Plotting with pandas and plotly

In [ ]:
lidar = lidar1

In [ ]:
df_daq = lidar.file_handler.get_pandas_daq()
df_gimbal = lidar.file_handler.get_pandas_gimbal()
df_laser = lidar.file_handler.get_pandas_laser()
df_tot = lidar.file_handler.get_pandas_tot()
df_counts = lidar.file_handler.get_pandas_counts()
df_measurement = lidar.file_handler.get_pandas_measurement()

df_dcounts = lidar.pds.get_pandas_dcounts()
df_rates = lidar.pds.get_pandas_rate()

In [ ]:
df_rates[df_rates['rate_pmt']!=0]

# Basic plots direct from file

In [ ]:
fig = multiline_plot(df_daq)
fig.show()

fig = multiline_plot(df_gimbal)
fig.show()

fig = multiline_plot(df_laser)
fig.show()

fig = multiline_plot(df_measurement)
fig.show()

fig = multiline_plot(df_rates[df_rates['rate_pmt']!=0])
fig.show()

# Check if measurement steps is valid, i.e.: [0,0,1,1,...,i,i]

In [ ]:
step_r = lidar.file_handler.measurement_step[:].reshape(-1,2)
time_r = lidar.file_handler.measurement_time[:].reshape(-1,2)

if np.any(step_r[:,0] != step_r[:,1]):
    raise ValueError('corrupt data! step_r[:,0] must be step_r[:,1].')

# Check DAQ if daq is working properly
When step is up, here laser_set_adjust_x.max(), there should be no change of the laser_set_adjust_x/y.

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=lidar.file_handler.laser_time,
                         y=lidar.file_handler.laser_set_adjust_x,
                         name='adjust_x'))
fig.add_trace(go.Scatter(x=lidar.file_handler.laser_time,
                         y=lidar.file_handler.laser_set_adjust_y,
                         name='adjust_y'))

# create a step_line which is high when measurement_step active and low when inactive
t_one_off= np.array([lidar.file_handler.measurement_time,
                     lidar.file_handler.measurement_time]).T

one_off = np.zeros_like(t_one_off)
one_off[::2] = [lidar.file_handler.laser_set_adjust_x[:].min(), lidar.file_handler.laser_set_adjust_x[:].max()]
one_off[1::2] = [lidar.file_handler.laser_set_adjust_x[:].max(), lidar.file_handler.laser_set_adjust_x[:].min()]

fig.add_trace(go.Scatter(x=t_one_off.flatten(),
                         y=one_off.flatten(),
                         name='steps'))


# Calculate the values of interest from the scan-measurements

In [ ]:
# the absolute timestamp from the middle of the reading
abs_timestamp_middle = lidar.file_handler.counts_time[:-1] + lidar.pds.rate_time*.5

bin_counts_pmt, bin_edges, binnumber = scipy.stats.binned_statistic(
    abs_timestamp_middle,
    lidar.pds.dcounts_pmt,
    statistic='sum',
    bins=lidar.file_handler.measurement_time)

bin_counts_laser, bin_edges, binnumber = scipy.stats.binned_statistic(
    abs_timestamp_middle,
    lidar.pds.dcounts_laser,
    statistic='sum',
    bins=lidar.file_handler.measurement_time)

# take the middle of the measurement step-time and get the corresponding adjust_x/y values
measurement_time = lidar.file_handler.measurement_time[:-1] + np.diff(lidar.file_handler.measurement_time)*.5
measurement_adjust_x = np.interp(lidar.file_handler.measurement_time,
                                 lidar.file_handler.laser_time,
                                 lidar.file_handler.laser_set_adjust_x)

measurement_adjust_y = np.interp(lidar.file_handler.measurement_time,
                                 lidar.file_handler.laser_time,
                                 lidar.file_handler.laser_set_adjust_y)

In [ ]:
# check, should be all ints. If not DAQ isn't working properly.
for i in zip(measurement_adjust_x[::2], measurement_adjust_y[::2]):
    print(i)

## Store the values of interest

In [ ]:
df_scan = pandas.DataFrame({'success_rate': bin_counts_pmt/bin_counts_laser,
                            'adjust_x': np.round(measurement_adjust_x[:-1],0).astype(int),
                            'adjust_y': np.round(measurement_adjust_y[:-1],0).astype(int)})

# df_scan = df_scan[::2]  # cut when moving laser

# for 3D and Contour plots
x = np.unique(df_scan['adjust_x'])
y = np.unique(df_scan['adjust_y'])
zz = np.zeros((x.shape[0], y.shape[0]) , dtype=float)
zz_err = np.zeros((x.shape[0], y.shape[0]) , dtype=float)


for i, x_i in enumerate(x):
    for j, y_j in enumerate(y):
        mask = (df_scan['adjust_x'].astype(int) == x_i) & (df_scan['adjust_y'].astype(int) == y_j)
        zz[i,j] = np.mean(df_scan['success_rate'][mask])
        zz_err[i,j] = np.std(df_scan['success_rate'][mask])
        
dict_scan_3d = {'success_rate': zz,
                'success_err': zz_err,
                'adjust_x': x,
                'adjust_y': y}

# 2D scatter with scan track

In [ ]:
go.scatter.Line
fig = go.Figure(data=[
    go.Scatter(x=df_scan['adjust_x'],
               y=df_scan['adjust_y'], 
               mode='lines+markers',
               line=dict(color='gray'),
               marker=dict(color=df_scan['success_rate'],
                           size=df_scan['success_rate']*2+10,
                           colorscale='Viridis',
                           colorbar=dict(title="Events/Trigger"),
                          ),
              ),
                     ])
fig.update_layout(xaxis_title="Adjust X Steps [a.u.]",
                  yaxis_title="Adjust Y Steps [a.u.]",
                  width=600, height=500,
                  margin=dict(l=0, r=0, b=0, t=0),
                 )
fig.show()

In [ ]:
go.scatter.Line
fig = go.Figure(data=[
    go.Scatter3d(x=df_scan['adjust_x'],
                 y=df_scan['adjust_y'], 
                 z=df_scan['success_rate'],
                 mode='lines+markers',
                 line=dict(color='gray'),
                 marker=dict(color=df_scan['success_rate'],
                             size=df_scan['success_rate']*2+10,
                             colorscale='Viridis',
                             colorbar=dict(title="Events/Trigger"),
                            )
                 ),
                     ])

fig.update_layout(scene = dict(
                    xaxis_title='Adjust X Steps [a.u.]',
                    yaxis_title='Adjust Y Steps [a.u.]',
                    zaxis_title='Rate [Events/Trigger]'),
                  width=900,
                  height=700,
                  margin=dict(r=0, l=0, b=0, t=0)
                 )

fig.show()

# 3D plot

In [ ]:
fig = go.Figure(data=[go.Surface(x=dict_scan_3d['adjust_x'],
                                 y=dict_scan_3d['adjust_y'],
                                 z=dict_scan_3d['success_rate'],
                                 colorscale='Viridis',
                                 colorbar=dict(title="Events/Trigger"),
                                )
                     ]
               )

fig.update_layout(autosize=False,
#                   width=500, height=500,
#                   margin=dict(l=65, r=50, b=65, t=90),
                 scene = dict(
                    xaxis_title='Adjust X Steps [a.u.]',
                    yaxis_title='Adjust Y Steps [a.u.]',
                    zaxis_title='Rate [Events/Trigger]'),
                  width=900,
                  height=700,
                  margin=dict(r=0, l=0, b=0, t=0),
                 )


fig.show()

In [ ]:
fig = go.Figure(data=[go.Contour(x=dict_scan_3d['adjust_x'],
                                 y=dict_scan_3d['adjust_y'],
                                 z=dict_scan_3d['success_rate'],
                                 colorscale='viridis',
                                 colorbar=dict(title="Events/Trigger")
                                ),
                     ]
               )

fig.update_layout(autosize=False,
                  xaxis_title="Adjust X Steps [a.u.]",
                  yaxis_title="Adjust Y Steps [a.u.]",
                  width=600, height=500,
                  margin=dict(l=0, r=0, b=0, t=0),
                 )

fig.show()

fig = go.Figure(data=[go.Contour(x=dict_scan_3d['adjust_x'],
                                 y=dict_scan_3d['adjust_y'],
                                 z=dict_scan_3d['success_err'],
                                 colorscale='viridis',
                                 colorbar=dict(title="STD Events/Trigger")
                                ),
                     ]
               )

fig.update_layout(autosize=False,
                  xaxis_title="Adjust X Steps [a.u.]",
                  yaxis_title="Adjust Y Steps [a.u.]",
                  width=600, height=500,
                  margin=dict(l=0, r=0, b=0, t=0),
                 )

fig.show()